In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# --- Étape 1: Filtrage des Données ---

# 1. Identifier et conserver les utilisateurs qui ont noté au moins 200 livres
user_counts = df_ratings['user'].value_counts()
popular_users = user_counts[user_counts >= 200].index
df_ratings_filtered = df_ratings[df_ratings['user'].isin(popular_users)]

# 2. Identifier et conserver les livres qui ont au moins 100 notes
book_counts = df_ratings_filtered['isbn'].value_counts()
popular_books = book_counts[book_counts >= 100].index
df_ratings_final = df_ratings_filtered[df_ratings_filtered['isbn'].isin(popular_books)]
# --- Étape 2: Création de la Matrice Livre par Utilisateur ---
# Fusionner les notes filtrées avec les titres
df_merged = df_ratings_final.merge(df_books, on='isbn')
# Supprimer les notes dupliquées
df_pivot = df_merged.drop_duplicates(subset=['user', 'title'])
# Créer la table pivot (Livre x Utilisateur) et remplir les cases vides par 0
book_pivot = df_pivot.pivot_table(
    index='title',
    columns='user',
    values='rating'
).fillna(0)
# Convertir en Matrice Creuse (pour optimiser le calcul KNN)
from scipy.sparse import csr_matrix
book_sparse_matrix = csr_matrix(book_pivot.values)


# --- Étape 3: Entraînement du Modèle KNN ---
from sklearn.neighbors import NearestNeighbors
# Initialiser et entraîner le modèle
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(book_sparse_matrix)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):

    # 1. Vérification
    if book not in book_pivot.index:
        return [book, []]

    # 2. Trouver l'index du livre cible
    book_index = book_pivot.index.get_loc(book)

    # 3. Trouver les 6 voisins (le livre + 5 recommandations)
    distances, indices = model_knn.kneighbors(
        book_sparse_matrix[book_index].reshape(1, -1),
        n_neighbors=6
    )

    recommended_books = [book]
    recommendations_list = []

    # 4. Construire la liste de formatage (on commence à l'index 1 pour sauter le livre lui-même)
    for i in range(1, len(distances.flatten())):
        distance = distances.flatten()[i]
        title = book_pivot.index[indices.flatten()[i]]
        recommendations_list.append([title, distance])

    # Inverser la liste pour que le livre le plus proche soit en haut (critère du test)
    recommendations_list.reverse()
    recommended_books.append(recommendations_list)

    return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()